In [44]:
import json
import gzip
import pandas as pd
import numpy as np


In [45]:
df_steam_games = pd.read_json(
    'datasets\output_steam_games.json', lines=True
)

In [46]:
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [47]:
df_steam_games.dropna(how="all", inplace=True)

In [48]:
duplicados = df_steam_games[df_steam_games["id"].duplicated()]


In [49]:
df_steam_games.drop_duplicates(subset="id",inplace=True)
df_steam_games.dropna(subset="id",inplace=True)



In [50]:
df_steam_games[["app_name", "title"]]
df_steam_games['title'] = df_steam_games['title'].fillna(df_steam_games['app_name'])

## Trabajo con la columna price

In [51]:
df_steam_games["price"].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, None, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

In [52]:
df_steam_games["price"].isna().sum()


1377

In [53]:
mask = []
for i in df_steam_games["genres"]:
    if i is None:
        mask.append(False)
        continue
    elif "Free to play" in i:
        mask.append(True)
    else:
        mask.append(False)

df_steam_games.loc[(mask) & (df_steam_games["price"].isna()), "price"] = 0.0

In [54]:
df_steam_games["price"].isna().sum()

1377

In [55]:
df_steam_games.dropna(subset="price", inplace=True)

In [56]:
df_steam_games["price"] = df_steam_games["price"].astype(str)


In [57]:
df_steam_games = df_steam_games.replace(to_replace=".*Free.*", value=0, regex=True)

In [58]:
df_steam_games["price"].unique()

array(['4.99', 0, '0.99', '2.99', '3.99', '9.99', '18.99', '29.99',
       '10.99', '1.5899999999999999', '14.99', '1.99', '59.99', '8.99',
       '6.99', '7.99', '39.99', '19.99', '7.49', '12.99', '5.99', '2.49',
       '15.99', '1.25', '24.99', '17.99', '61.99', '3.49', '11.99',
       '13.99', '34.99', '74.76', '1.49', '32.99', '99.99', '14.95',
       '69.99', '16.99', '79.99', '49.99', '5.0', '44.99', '13.98',
       '29.96', '119.99', '109.99', '149.99', '771.71', 'Install Now',
       '21.99', '89.99', 'Play WARMACHINE: Tactics Demo', '0.98',
       '139.92', '4.29', '64.99', '54.99', '74.99', 'Install Theme',
       '0.89', 'Third-party', '0.5', 'Play Now', '299.99', '1.29', '3.0',
       '15.0', '5.49', '23.99', '49.0', '20.99', '10.93',
       '1.3900000000000001', '36.99', '4.49', '2.0', '4.0', '9.0',
       '234.99', '1.9500000000000002', '1.5', '199.0', '189.0', '6.66',
       '27.99', '10.49', '129.99', '179.0', '26.99', '399.99', '31.99',
       '399.0', '20.0', '40.0', 

In [59]:
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
df_steam_games['price'] = df_steam_games['price'].fillna(0.0)

In [60]:
df_steam_games['price'] = df_steam_games['price'].astype(float).round(2)


In [61]:
len(df_steam_games)

30755

## Trabajo con la columna release_date

In [62]:
df_steam_games['release_date'].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2004-03-16',
       '2004-03-01', '2016-11-19'], dtype=object)

In [63]:
df_steam_games['release_date'].isna().sum()

1936

In [64]:
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')
df_steam_games['release_date'] = df_steam_games['release_date'].dt.year

In [65]:
df_steam_games.dropna(subset="release_date", inplace=True)


In [66]:
df_steam_games['release_date'].unique()

array([2018., 2017., 1997., 1998., 2016., 2006., 2005., 2003., 2007.,
       2002., 2000., 1995., 1996., 1994., 2001., 1993., 2004., 2008.,
       2009., 1999., 1992., 1989., 2010., 2011., 2013., 2012., 2014.,
       1983., 1984., 2015., 1990., 1988., 1991., 1985., 1982., 1987.,
       1981., 1986., 2021., 2019., 1975., 1970., 1980.])

In [67]:
len(df_steam_games)

28780

## Trabajo con la columna genres

In [68]:
df_steam_games['genres'].isna().sum()

1203

In [69]:
#df_steam_games[["genres", "tags"]]
df_steam_games['genres'].fillna(df_steam_games['tags'], inplace=True)

In [70]:
df_steam_games['genres'].isna().sum()

116

In [71]:
df_steam_games.dropna(subset="genres", inplace=True)

In [72]:
df_steam_games['genres'].value_counts()

genres
[Action]                                                                                                        1808
[Action, Indie]                                                                                                 1572
[Simulation]                                                                                                    1372
[Casual, Simulation]                                                                                            1357
[Strategy]                                                                                                      1020
                                                                                                                ... 
[Casual, Indie, Sports, Strategy, Early Access]                                                                    1
[Action, Free to Play, Simulation]                                                                                 1
[Animation &amp; Modeling, Audio Production, Design &amp;

In [73]:
len(df_steam_games)

28664

## Trabajo con la columna developer

In [74]:
df_steam_games['developer'].isna().sum()


1203

In [75]:
df_steam_games['developer'].fillna(df_steam_games['publisher'], inplace=True)

In [76]:
df_steam_games.dropna(subset="developer", inplace=True)

In [77]:
len(df_steam_games)

27521

In [78]:
df_steam_games["release_date"] = df_steam_games["release_date"].astype(int)


In [79]:
df_steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd


### Creo la nueva tabla ya limpia, con las columnas de interes

In [80]:
df_steam_games_final = df_steam_games.drop(["publisher", "app_name", "url", "tags", "reviews_url", "specs", "early_access"], axis=1)

In [81]:
df_steam_games_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27521 entries, 88310 to 120443
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        27521 non-null  object 
 1   title         27521 non-null  object 
 2   release_date  27521 non-null  int32  
 3   price         27521 non-null  float64
 4   id            27521 non-null  float64
 5   developer     27521 non-null  object 
dtypes: float64(2), int32(1), object(3)
memory usage: 1.4+ MB


### Messi rve

In [82]:
df_steam_games_final.to_csv('df_steam_games_clean.csv', sep=',' , index=False)
#df = pd.read_csv('df_steam_games_clean.csv')
#df = pd.read_csv('df_steam_games_clean.csv')

In [83]:
df_steam_games_clean = pd.read_csv('df_steam_games_clean.csv')

In [84]:
df_steam_games_clean

,genres,title,release_date,price,id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018,0.00,643980.0,Secret Level SRL
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017,0.00,670290.0,Poolians.com
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017,0.99,767400.0,彼岸领域
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018,3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...
27516,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018,1.99,745400.0,Bidoniera Games
27517,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018,1.99,773640.0,"Nikita ""Ghost_RUS"""
27518,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018,4.99,733530.0,Sacada
27519,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018,1.99,610660.0,Laush Dmitriy Sergeevich


In [85]:
df_steam_games_clean["release_date"] = df_steam_games_clean["release_date"].astype(int)

In [86]:
df_steam_games_clean

,genres,title,release_date,price,id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018,4.99,761140.0,Kotoshiro
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018,0.00,643980.0,Secret Level SRL
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017,0.00,670290.0,Poolians.com
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017,0.99,767400.0,彼岸领域
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018,3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...
27516,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018,1.99,745400.0,Bidoniera Games
27517,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018,1.99,773640.0,"Nikita ""Ghost_RUS"""
27518,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018,4.99,733530.0,Sacada
27519,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018,1.99,610660.0,Laush Dmitriy Sergeevich


for i in range(0, len(df_steam_games["genres"])):
    if df_steam_games["genres"].iloc[i] != None:
        for j in df_steam_games["genres"].iloc[i]:
            if "Free" in j:
                print(df_steam_games["genres"].iloc[i])
                print(df_steam_games["price"].iloc[i])
                print(df_steam_games["url"].iloc[i])
                print("-" * 50) 

In [87]:


#df_steam_games["release_date"].isna().sum()
#print(final_df[final_df.isnull().any(axis=1)])
#final_df[final_df["title"].isnull()]

#final_df.loc[
  #  final_df['title'] == 'Batman: Arkham City - Game of the Year Edition', 'id'
  #  ] = final_df['url'].apply(lambda x: x[-6:])

#linea = final_df.loc[88310]
#linea
#final_df["title"].isna().sum()

#final_df[(final_df['title'].isna()) & (final_df['app_name'].isna())]

#df_steam_games.dropna(subset="price", inplace=True)

#df_steam_games['genres'].apply(print)


import json
import gzip
import pandas as pd
import numpy as np

compressed_file = 'Datasets/output_steam_games.json'


with gzip.open(compressed_file, 'rb') as f:
    data = f.read()


json_str = data.decode('utf-8')
data_list = json_str.strip().split('\n')


dfs = []
for item in data_list:
    data_dict = json.loads(item)
    df = pd.DataFrame([data_dict])
    dfs.append(df)


final_df = pd.concat(dfs, ignore_index=True)